# Load and Validate Raw Data from AWS S3
This notebook was to  focus on reading the raw data from S3, inspecting the dataset, and validating its contents but 
the platform wasnt available at the time . Therefore we use Local File sysytem

Recommendation:
If scalability is important:

Use Google Cloud Storage (GCS) or Azure Data Lake (ADLS) for a seamless cloud alternative to AWS S3. If working locally or on a small project:
Use the local file system or MinIO for simple and lightweight storage.

In [4]:
import os
os.environ["HADOOP_HOME"] = "C:\\hadoop"
os.environ["PATH"] += os.pathsep + "C:\\hadoop\\bin"


In [6]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, when, isnull

# Initialize Spark session
spark = SparkSession.builder \
    .appName("CustomerChurnDataIngestion") \
    .config("spark.hadoop.fs.file.impl.disable.cache", "true") \
    .getOrCreate()


# Load raw customer data from the local file system
file_path = r"C:\Users\ADMIN\Desktop\Projects\Batch-Processing-Project-Customer-Churn-Prediction-Pipeline\datasets\Bank Customer Churn Prediction.csv"
raw_data = spark.read.csv(f"file:///{file_path.replace('\\', '/')}", header=True, inferSchema=True)

# Display the first few rows of the raw data
print("Raw Data Sample:")
raw_data.show(5)


Raw Data Sample:
+-----------+------------+-------+------+---+------+---------+---------------+-----------+-------------+----------------+-----+
|customer_id|credit_score|country|gender|age|tenure|  balance|products_number|credit_card|active_member|estimated_salary|churn|
+-----------+------------+-------+------+---+------+---------+---------------+-----------+-------------+----------------+-----+
|   15634602|         619| France|Female| 42|     2|      0.0|              1|          1|            1|       101348.88|    1|
|   15647311|         608|  Spain|Female| 41|     1| 83807.86|              1|          0|            1|       112542.58|    0|
|   15619304|         502| France|Female| 42|     8| 159660.8|              3|          1|            0|       113931.57|    1|
|   15701354|         699| France|Female| 39|     1|      0.0|              2|          0|            0|        93826.63|    0|
|   15737888|         850|  Spain|Female| 43|     2|125510.82|              1|         

In [7]:
# Check schema of the loaded data

print("Schema of Raw Data:")
raw_data.printSchema()




Schema of Raw Data:
root
 |-- customer_id: integer (nullable = true)
 |-- credit_score: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- balance: double (nullable = true)
 |-- products_number: integer (nullable = true)
 |-- credit_card: integer (nullable = true)
 |-- active_member: integer (nullable = true)
 |-- estimated_salary: double (nullable = true)
 |-- churn: integer (nullable = true)



In [8]:
# Validate the data (e.g., check for missing values)

print("Missing Data Check:")
missing_data = raw_data.select([count(when(isnull(c), c)).alias(c) for c in raw_data.columns])
missing_data.show()

# Check for duplicate rows
total_rows = raw_data.count()
unique_rows = raw_data.distinct().count()
duplicates = total_rows - unique_rows

print(f"Total Rows: {total_rows}")
print(f"Unique Rows: {unique_rows}")
print(f"Number of Duplicate Rows: {duplicates}")



Missing Data Check:
+-----------+------------+-------+------+---+------+-------+---------------+-----------+-------------+----------------+-----+
|customer_id|credit_score|country|gender|age|tenure|balance|products_number|credit_card|active_member|estimated_salary|churn|
+-----------+------------+-------+------+---+------+-------+---------------+-----------+-------------+----------------+-----+
|          0|           0|      0|     0|  0|     0|      0|              0|          0|            0|               0|    0|
+-----------+------------+-------+------+---+------+-------+---------------+-----------+-------------+----------------+-----+

Total Rows: 10000
Unique Rows: 10000
Number of Duplicate Rows: 0


In [ ]:
# Save the validated data for the next step (optional, as Parquet format)
#output_path = r"C:\Users\ADMIN\Desktop\Projects\Batch-Processing-Project-Customer-Churn-Prediction-Pipeline\datasets\validated_data.parquet"
output_path = "file:///C:/Users/ADMIN/Desktop/Projects/Batch-Processing-Project-Customer-Churn-Prediction-Pipeline/datasets/validated_data.parquet"

#raw_data.write.parquet(f"file:///{output_path.replace('\\', '/')}", mode="overwrite")
raw_data.write.parquet(output_path, mode="overwrite")


# Stop Spark session
spark.stop()

In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Customer Churn Prediction") \
    .config("spark.hadoop.fs.file.impl.disable.cache", "true") \
    .getOrCreate()
